In [106]:
import pandas as pd
import json
import re

In [107]:
with open('/Users/nhotin/Documents/GitHub/LegalBizAI_project/data/documents/process/qaset_rd.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
df = pd.json_normalize(data)

In [108]:
def process_references(references):
    if not references:
        return None
    processed_refs = []
    for ref in references:
        # Tìm vị trí của từ "Điều" theo sau là số
        match = re.search(r'Điều \d+.*', ref)
        if match:
            # Giữ lại phần từ "Điều" trở về sau
            processed_refs.append(match.group(0))
    return processed_refs if processed_refs else None

In [109]:
def split_references(references):
    if not references:
        return None
    split_refs = []
    for ref in references:
        parts = re.findall(r'(Điều \d+)', ref)
        if parts:
            for part in parts:
                law = ref.replace(part, '').strip(', ').strip()
                if law:
                    split_refs.append([part, law])
                else:
                    split_refs.append([part])
    return split_refs if split_refs else None

In [110]:
df['references'] = df['references'].apply(process_references)
df['references'] = df['references'].apply(split_references)

In [111]:
df = df[df['references'].map(lambda x: x != [])]
df = df[df['references'].notna()]
df.dropna(axis=1, how='all', inplace=True)

In [112]:
df.head()

,question,answer,references
0,Hội đồng giải thể doanh nghiệp do Nhà nước nắm...,Theo quy định tại Điều 44 Nghị định 23/2022/NĐ...,"[[Điều 44, Nghị định 23/2022/NĐ-CP]]"
1,Có được quyền kéo dài thêm thời hạn giải thể d...,Theo quy định tại Điều 47 Nghị định 23/2022/NĐ...,"[[Điều 47, Nghị định 23/2022/NĐ-CP]]"
2,Khi giải thể doanh nghiệp do Nhà nước nắm giữ ...,Theo quy định tại khoản 2 Điều 39 Nghị định 23...,"[[Điều 39, Nghị định 23/2022/NĐ-CP]]"
3,Hồ sơ giải thể doanh nghiệp mới nhất năm 2023 ...,Căn cứ theo quy định tại khoản 1 Điều 210 Luật...,"[[Điều 210, Luật Doanh nghiệp 2020]]"
4,Thứ tự thanh toán khoản nợ của doanh nghiệp gi...,Theo quy định tại khoản 5 Điều 208 Luật Doanh ...,"[[Điều 208, Luật Doanh nghiệp 2020]]"


In [113]:
with open('/Users/nhotin/Documents/GitHub/LegalBizAI_project/data/documents/process/article/qaset_article.json', 'w', encoding='utf-8') as file:
    json.dump(df.to_dict(orient='records'), file, ensure_ascii=False, indent=4)